In [ ]:
# set module
import pandas as pd


from google.colab import drive
drive.mount('/content/drive/')


#경로 설정
import os

new_dir = "/content/drive/MyDrive/기상청/데이터/지면온도"
os.chdir(new_dir)  # 현재 작업 디렉토리 변경
current_dir = os.getcwd()
print("현재 작업 디렉토리:", current_dir)



In [ ]:
# 데이터 읽어오기
data_train = pd.read_csv("./surface_tp_train.csv")
data_test= pd.read_csv("./surface_tp_test.csv")

In [ ]:
## column 명 추출

data_train.drop('Unnamed: 0', axis = 1, inplace = True)
data_train.drop('...1', axis = 1, inplace = True)
data_test.drop('Unnamed: 0', axis = 1, inplace = True)


In [ ]:
data_train.columns


In [ ]:
data_test.columns


## 시각화 진행

In [ ]:
## 대푯값 확인 -> train
data_train.describe()

In [ ]:
data_train.info()

In [ ]:

## 제거 : -99, -99.9, -999

import pandas as pd
import numpy as np

class data_preprocessing():
  def __init__(self, data, test):
    self.data = data
    self.na_counts = None
    self.list_na = None
    self.ratio_result = None
    self.list_columns = None
    self.test = test

    # 99, 99.9 값을 null로 바꾸는 함수 정의
  def replace_values_with_null(self, value):
    if value == -99 or value == -99.9 or value == -999:
        return np.nan
    else:
        return value

  def replace_99(self): ## 이 함수를 실행하면 작동
    self.data = self.data.applymap(self.replace_values_with_null)
    return self.data


  ## na 개수 발견
  def count_na_per_column(self):
    list_na = []
    for i in self.data.columns:
      self.na_counts = self.data[i].isna().sum()
      list_na.append(self.na_counts)

    self.list_na = list_na
    return self.list_na

  def divide_list_elements(self, lst, divisor):
      divided_lst = [x / divisor for x in lst]
      return divided_lst

  ## na 개수 전체 데이터 중 비율 세기
  def ratio(self):
    self.ratio_result = self.divide_list_elements(self.list_na, len(self.data))
    return self.ratio_result

  def remove_column(self):
    self.list_columns = []
    # 역순으로 반복문 실행하여 열을 삭제
    for i in range(len(self.ratio_result)-1, -1, -1):
        if self.ratio_result[i] >= 0.4:
          self.list_columns.append(self.data.columns[i])
            # print(self.ratio_result[i])
            # print(i)
          self.data = self.data.drop(self.data.columns[i], axis=1)
    return self.data

  def remove_columns_test(self):
    data = self.data
    for i in self.list_columns:
      ## . 기준으로 마지막 부분 없애고,
      new_col = i.replace("train", "test")
      # print(new_col)
      try: ## train만 있는 열이 있을 경우에는 pass 하자
        data = data.drop(new_col, axis=1)
      except:
        pass
    return data


  def result(self, train_or_test):
    if train_or_test == "train":
      self.replace_99()
      self.count_na_per_column()
      self.ratio()
      return self.remove_column()
    else:
      self.data = self.test
      self.replace_99() ## null 로 변경
      return self.remove_columns_test() ## train에서 제거되는 행으로 제거






In [ ]:
dp= data_preprocessing(data_train, data_test)
result_train = dp.result('train')
result_train
result_test = dp.result("test")
result_test

In [ ]:
result_train

In [ ]:
result_train.dropna(inplace = True)
result_train.info()

In [ ]:
# result_train['hh'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[-2:]
# result_train['dd'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[-4:-2]
# result_train['mm'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[:-4]
# year_mapping = {'A': 2001, 'B': 2002, 'C': 2003, 'D': 2004, 'E': 2005, 'F': 2006}
# result_train['new_year'] = result_train['surface_tp_train.year'].map(year_mapping)
# result_train['mm'] = result_train['mm'].apply(lambda x: x.zfill(2))
# result_train['date'] = pd.to_datetime(result_train['new_year'].astype(str) + result_train['mm'] + result_train['dd'] + result_train['hh'], format='%Y%m%d%H')

def adding_date(data_original, season): # season: all, spring, summer, fall, winter 계절별로 넣기 가능
  data = data_original
  data['hh'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[-2:]
  data['dd'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[-4:-2]
  data['mm'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[:-4]
  year_mapping = {'A': 2001, 'B': 2002, 'C': 2003, 'D': 2004, 'E': 2005, 'F': 2006}
  data['new_year'] = data['surface_tp_train.year'].map(year_mapping)
  data['mm'] = data['mm'].apply(lambda x: x.zfill(2))
  data['date'] = pd.to_datetime(data['new_year'].astype(str) + data['mm'] + data['dd'] + data['hh'], format='%Y%m%d%H')

  spring_month_list = ['02', '03', '04']
  summer_month_list = ['05', '06', '07']
  fall_month_list = ['08', '09', '10']
  winter_month_list = ['11', '12', '01']

  if season == "all":
    pass
  elif season == "spring":
    filtered_data = data[data['mm'].isin(spring_month_list)]
  elif season == "summer":
    filtered_data = data[data['mm'].isin(summer_month_list)]
  elif season == "fall":
    filtered_data = data[data['mm'].isin(fall_month_list)]
  elif season == "winter":
    filtered_data = data[data['mm'].isin(winter_month_list)]
  else:
    raise ValueError("not season value")

  # data = filtered_data.drop(["surface_tp_train.year", 'surface_tp_train.mmddhh', 'hh', 'dd', 'mm', 'new_year'], axis = 1)
  data = filtered_data.drop(["surface_tp_train.year", 'surface_tp_train.mmddhh'], axis = 1)
  # data["datetime"] = pd.to_datetime(data["date"])  # 문자열을 날짜/시간 유형으로 변환
  ## datetime 순으로 index 생성
  data.reset_index(inplace=True)
  data.index = data.index.astype(int)
  return data

### 지역 별로 데이터를 쪼개주는 함수

def region_category(data, region_number): # region_number는 자기가 알아서 넣기로

  # 주어진 열을 기준으로 데이터 프레임을 필터링하고 값 종류별로 데이터를 쪼갬
  unique_values = data['surface_tp_train.stn'].unique()  # 주어진 열의 고유한 값들을 추출

  if region_number in unique_values:
    result = data[data['surface_tp_train.stn'] == region_number]  # 주어진 값에 해당하는 데이터만 필터링하여 저장
  else:
    raise ValueError("not in region number")

  return result


def change_columns(data):
  data.columns = data.columns.str.replace(".", "_")


def change_to_categrory(data, columns_name):
  data[columns_name] = data[columns_name].astype(str).astype('category')
  return data


In [ ]:
result_train_spring = adding_date(result_train, "spring")
result_train_spring.columns


## region 별로 나누기
train_region_1_spring = region_category(result_train_spring, 1)
train_region_2_spring = region_category(result_train_spring, 2)
train_region_3_spring = region_category(result_train_spring, 3)
train_region_4_spring = region_category(result_train_spring, 4)
train_region_5_spring = region_category(result_train_spring, 5)
train_region_6_spring = region_category(result_train_spring, 6)
train_region_7_spring = region_category(result_train_spring, 7)
train_region_8_spring = region_category(result_train_spring, 8)
train_region_9_spring = region_category(result_train_spring, 8)
train_region_10_spring = region_category(result_train_spring, 10)

change_columns(result_train_spring)

In [ ]:
list_colunms = ["surface_tp_train_stn", "new_year", "mm", "dd", "hh","surface_tp_train_ww"]

# train_region_1_spring = change_to_categrory(train_region_1_spring, "surface_tp_train_stn")
# train_region_1_spring = change_to_categrory(train_region_1_spring, "new_year")
# train_region_1_spring = change_to_categrory(train_region_1_spring, "mm")
# train_region_1_spring = change_to_categrory(train_region_1_spring, "dd")
# train_region_1_spring = change_to_categrory(train_region_1_spring, "hh")
# train_region_1_spring = change_to_categrory(train_region_1_spring, "hh")
# train_region_1_spring = change_to_categrory(train_region_1_spring, "surface_tp_train_ww")

result_train_spring = change_to_categrory(result_train_spring, "surface_tp_train_stn")
result_train_spring = change_to_categrory(result_train_spring, "new_year")
result_train_spring = change_to_categrory(result_train_spring, "mm")
result_train_spring = change_to_categrory(result_train_spring, "dd")
result_train_spring = change_to_categrory(result_train_spring, "hh")
result_train_spring = change_to_categrory(result_train_spring, "hh")
result_train_spring = change_to_categrory(result_train_spring, "surface_tp_train_ww")



## column 이름에 . 제거
change_columns(result_train_spring)
result_train_spring = result_train_spring.drop("index", axis = 1)
result_train_spring.info()

## sort 하고 타임 index 생성
result_train_spring.sort_values("date", ascending = True, inplace = True)
result_train_spring['time_idx'] = range(len(result_train_spring))

result_train_spring.head(5)
result_train_spring.columns

In [ ]:
result_train_spring = result_train_spring.drop("date", axis = 1)
result_train_spring = result_train_spring.drop("new_year", axis = 1)

## Temporal Fusion Transformer 모델링 진행

In [ ]:
## Temporal Fusion Transformer(TFT) 모델 다운로드
! pip install pytorch_forecasting

In [ ]:

new_directory = "/content/drive/MyDrive/기상청 (1)"
os.chdir(new_directory)
root_directory = os.getcwd()
root_directory

In [ ]:

# ## Temporal Fusion Transformer(TFT) 모델


# #경로만 자신의 환경에 맞게 잘 설정해주세요!
# DATAROOT= root_directory + '/tft'
# CKPTROOT = DATAROOT+"/batch_128/ckpts" # directory for model checkpoints
# CSVROOT = DATAROOT+"/batch_128/csvs" # directory for prediction outputs
# SUBFN = DATAROOT+"/batch_128/sub.csv" # final submission file path
# LOGDIR = DATAROOT+"/batch_128/logs" # pytorch_forecasting requirs logger

# import sys
# import os
# import argparse
# import shutil
# import random
# from pathlib import Path

# import pandas as pd
# import numpy as np
# import torch
# import lightning.pytorch as pl

# # imports for training
# # import lightning.pytorch as pl

# from lightning.pytorch.loggers import TensorBoardLogger

# from pytorch_forecasting.data import (
#     TimeSeriesDataSet,
#     # GroupNormalizer
# )
# from lightning.pytorch.callbacks import (
#     ModelCheckpoint,
#     EarlyStopping,
#     LearningRateMonitor
# )

# # import dataset, network to train and metric to optimize
# from pytorch_forecasting import (
#     TimeSeriesDataSet,
#     TemporalFusionTransformer,
#     QuantileLoss
# )

# from pytorch_forecasting.metrics import SMAPE
# from pytorch_forecasting.models import TemporalFusionTransformer

# from lightning.pytorch.tuner import Tuner



# # number of epochs found in cv run
# NUM_EPOCHS = 200

# # number of seeds to use
# NUM_SEEDS = 10

# # batch_size
# BATCH_SIZE = 64


# # learning rate determined by a cv run with train data less 1 trailing week as validation
# LRS = [0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.05099279397234306 , 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
#        0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
#        0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
#        0.005099279397234306, 0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307,
#        0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307]





# # hyper parameters determined by cv runs with train data less 1 trailing week as validation
# PARAMS = {
#     'gradient_clip_val': 0.9658579636307634,
#     'hidden_size': 100,
#     'dropout': 0.19610151695402608,
#     'hidden_continuous_size': 90,
#     'attention_head_size': 4,
#     'learning_rate': 0.08
# }


# def seed_all(seed):
#     random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     np.random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     torch.backends.cudnn.deterministic = True
#     torch.backends.cudnn.benchmark = False



# def load_dataset(train_df, validate=False):
#     ENCODER_LENGTH_IN_WEEKS = 5
#     max_encoder_length = 24 * 7 *ENCODER_LENGTH_IN_WEEKS #5
#     max_prediction_length = 24 * 7
#     training_cutoff = train_df['time_idx'].max()-max_prediction_length #2040 - 24*7 = 1871

#     tr_ds = TimeSeriesDataSet(
#       train_df[lambda x: x.time_idx <=training_cutoff] if validate else train_df,
#       time_idx = "time_idx",
#       target = "surface_tp_train_ts", ## 예측값
#       group_ids=["surface_tp_train_stn"],
#       min_encoder_length = 1,
#       max_encoder_length = max_encoder_length,
#       min_prediction_length=1,
#       max_prediction_length=max_prediction_length,

#       #Known Inputs 알고 있는 변수
#       time_varying_known_categoricals = ['surface_tp_train_ww',
#                                          'hh',
#                                          'dd',
#                                          'mm',
#                                          'new_year'],
#       time_varying_known_reals = [
#         'surface_tp_train_ta',
#         'surface_tp_train_td',
#         'surface_tp_train_hm',
#         'surface_tp_train_ws',
#         'surface_tp_train_rn',
#         'surface_tp_train_re',
#         'time_idx'
#         ],

#       # target_normalizer=GroupNormalizer(groups=["surface_tp_train_stn"], transformation="softplus"),
#       #모르고 있는 변수
#       time_varying_unknown_categoricals=[],
#       time_varying_unknown_reals=[
#           "surface_tp_train_ts"
#         ],
#         add_relative_time_idx=True,  # add as feature
#         add_target_scales=True,  # add as feature
#         add_encoder_length=True,  # add as feature
#     )


#     va_ds = None
#     if validate:
#         va_ds = TimeSeriesDataSet.from_dataset(
#         tr_ds, train_df, predict=True, stop_randomization=True
#     )

#     return tr_ds, va_ds

# # training
# def fit(seed, tr_ds, va_loader=None):
#     seed_all(seed) # doesn't really work as training is non-deterministic

#     # create dataloaders for model
#     tr_loader = tr_ds.to_dataloader(
#         train=True, batch_size=BATCH_SIZE, num_workers=12
#     )
#     if va_loader is not None:
#         # stop training, when loss metric does not improve on validation set
#         early_stopping_callback = EarlyStopping(
#             monitor="val_loss",
#             min_delta=1e-4,
#             patience=20,
#             verbose=True,
#             mode="min"
#         )
#         lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
#         callbacks = [lr_logger, early_stopping_callback]
#     else:
#         # gather 10 checkpoints with best traing loss
#         checkpoint_callback = ModelCheckpoint(
#             monitor='train_loss',
#             dirpath=CKPTROOT,
#             filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
#             save_top_k=10
#         )
#         callbacks = [checkpoint_callback]

#     # create trainer
#     trainer = pl.Trainer(
#         max_epochs=66,
#         gradient_clip_val=PARAMS['gradient_clip_val'],
#         limit_train_batches=100,
#         callbacks=callbacks,
#         logger=TensorBoardLogger(LOGDIR),
#         # train_dataloaders = tr_loader
#         accelerator="gpu",
#     )

#     # use pre-deterined leraning rate schedule for final submission
#     learning_rate = LRS if va_loader is None else PARAMS['learning_rate']

#     # initialise model with pre-determined hyperparameters
#     tft = TemporalFusionTransformer.from_dataset(
#         tr_ds,
#         learning_rate=learning_rate,
#         hidden_size=PARAMS['hidden_size'],
#         attention_head_size=PARAMS['attention_head_size'],
#         dropout=PARAMS['dropout'],
#         hidden_continuous_size=PARAMS['hidden_continuous_size'],
#         output_size=1,
#         loss=SMAPE(), # SMAPE loss
#         log_interval=10,  # log example every 10 batches
#         logging_metrics=[SMAPE()],
#         reduce_on_plateau_patience=4,  # reduce learning automatically
#     )
#     print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")
#     tft = tft.to('cuda:0')

#     # kwargs = {'train_dataloaders': tr_loader}
#     # if va_loader:
#     #     kwargs['val_dataloaders'] = va_loader

#     # fit network
#     trainer.fit(
#         tft,
#         train_dataloaders = tr_loader,
#         val_dataloaders = va_loader,
#     )

#     best_model_path = trainer.checkpoint_callback.best_model_path
#     print(f'best_model_path={best_model_path}')
#     best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

#     return best_tft






In [ ]:
train_region_1_spring

In [ ]:
!nvidia-smi


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# predict 1 week
def forecast(ckpt, train_df, test_df):
    # load model
    best_tft = TemporalFusionTransformer.load_from_checkpoint(ckpt)
    max_encoder_length = best_tft.dataset_parameters['max_encoder_length']
    max_prediction_length = best_tft.dataset_parameters['max_prediction_length']

    assert max_encoder_length == 5*24*7 and max_prediction_length == 1*24*7

    # use 5 weeks of training data at the end
    encoder_data = train_df[lambda x: x.time_idx > x.time_idx.max() - max_encoder_length]

    # get last entry from training data
    last_data = train_df.iloc[[-1]]

    # fill NA target value in test data with last values from the train dataset
    target_cols = [c for c in test_df.columns if 'target' in c]
    for c in target_cols:
        test_df.loc[:, c] = last_data[c].item()

    decoder_data = test_df

    # combine encoder and decoder data. decoder data is to be predicted
    new_prediction_data = pd.concat([encoder_data, decoder_data], ignore_index=True)
    new_raw_predictions, new_x = best_tft.predict(new_prediction_data, mode="raw", return_x=True)

    # num_labels: mapping from 'num' categorical feature to index in new_raw_predictions['prediction']
    #             {'5': 4, '6': 6, ...}
    # new_raw_predictions['prediction'].shape = (60, 168, 1)
    num_labels = best_tft.dataset_parameters['categorical_encoders']['num'].classes_

    preds = new_raw_predictions['prediction'].squeeze()

    sub_df = pd.read_csv(DATAROOT+"/sample_submission.csv")

    # get prediction for each building (num)
    for n, ix in num_labels.items():
        sub_df.loc[sub_df.num_date_time.str.startswith(f"{n} "), 'answer'] = preds[ix].numpy()

    # save predction to a csv file
    outfn = CSVROOT+'/'+(Path(ckpt).stem + '.csv')
    print(outfn)
    sub_df.to_csv(outfn, index=False)

In [ ]:
# def ensemble(outfn):
#     # get all prediction csv files
#     fns = list(glob.glob(CSVROOT+"/*.csv"))
#     df0 = pd.read_csv(fns[0])
#     df = pd.concat([df0] + [pd.read_csv(fn).loc[:,'answer'] for fn in fns[1:]], axis=1)
#     # get median of all predcitions
#     df['median'] = df.iloc[:,1:].median(axis=1)
#     df = df[['num_date_time', 'median']]
#     df = df.rename({'median': 'answer'}, axis=1)
#     # save to submission file
#     df.to_csv(outfn, index=False)

# # not used for final submission
# def validate(seed, tr_ds, va_ds):
#     va_loader = va_ds.to_dataloader(
#         train=False, batch_size=BATCH_SIZE*10, num_workers=12
#     )
#     best_tft = fit(seed, tr_ds, va_loader)
#     actuals = torch.cat([y[0] for x, y in iter(va_loader)])
#     predictions = best_tft.predict(va_loader)
#     smape_per_num = SMAPE(reduction="none")(predictions, actuals).mean(1)
#     print(smape_per_num)
#     print(smape_per_num.mean())



# import glob
# print("### FORECAST ###")
# for p in glob.glob(CKPTROOT + "/*.ckpt"):
#     forecast(p, train_df, test_df)


# print("### ENSEMBLING ###")
# ensemble(SUBFN)

In [ ]:
## 정규화
# 스케일링을 하자
from sklearn.preprocessing import MinMaxScaler

def scale_columns(data):
    scaler = MinMaxScaler()
    scaled_data = data.copy()
    numeric_columns = scaled_data.select_dtypes(include=['float64', 'int64']).columns
    scaled_data[numeric_columns] = scaler.fit_transform(scaled_data[numeric_columns])
    return scaled_data

# 스케일링을 하자
from sklearn.preprocessing import RobustScaler
def robust_scaling(data):
    rbs = RobustScaler()
    scaled_data = data.copy()
    numeric_columns = ['surface_tp_train_ta',   'surface_tp_train_td',   'surface_tp_train_hm',   'surface_tp_train_ws',   'surface_tp_train_rn',   'surface_tp_train_re']
    scaled_data[numeric_columns] = rbs.fit_transform(scaled_data[numeric_columns])
    data[numeric_columns] = rbs.fit_transform(data[numeric_columns])
    return data

robust_scaling(result_train_spring)

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi


In [ ]:
## Temporal Fusion Transformer(TFT) 모델
import torch, gc
gc.collect()
torch.cuda.empty_cache()

#경로만 자신의 환경에 맞게 잘 설정해주세요!

## epoch = 40
DATAROOT= root_directory + '/tft/hidden_160/spring/final/epoch_40'
CKPTROOT = DATAROOT+"/batch_64/ckpts" # directory for model checkpoints
CSVROOT = DATAROOT+"/batch_64/csvs" # directory for prediction outputs
SUBFN = DATAROOT+"/batch_64/sub.csv" # final submission file path
LOGDIR = DATAROOT+"/batch_64/logs" # pytorch_forecasting requirs logger

import sys
import os
import argparse
import shutil
import random
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import lightning.pytorch as pl
import torchmetrics

from lightning.pytorch.callbacks import Callback



# imports for training
# import lightning.pytorch as pl

from lightning.pytorch.loggers import TensorBoardLogger

from pytorch_forecasting.data import (
    TimeSeriesDataSet,
    # GroupNormalizer
)
from lightning.pytorch.callbacks import (
    ModelCheckpoint,
    EarlyStopping,
    LearningRateMonitor
)

# import dataset, network to train and metric to optimize
from pytorch_forecasting import (
    TimeSeriesDataSet,
    TemporalFusionTransformer,
    QuantileLoss
)

from pytorch_forecasting.metrics import SMAPE
from pytorch_forecasting.models import TemporalFusionTransformer

from lightning.pytorch.tuner import Tuner

from pytorch_forecasting.metrics import MAE


# number of epochs found in cv run
NUM_EPOCHS = 40

# number of seeds to use
NUM_SEEDS = 10

# batch_size
BATCH_SIZE = 64

# learning rate determined by a cv run with train data less 1 trailing week as validation
LRS = [0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306, 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.05099279397234306 , 0.05099279397234306, 0.05099279397234306, 0.05099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.005099279397234306, 0.005099279397234306, 0.005099279397234306,
       0.005099279397234306, 0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307,
       0.0005099279397234307, 0.0005099279397234307, 0.0005099279397234307]

# hyper parameters determined by cv runs with train data less 1 trailing week as validation

## spring
PARAMS = {
    'gradient_clip_val': 0.9658579636307634,
    'hidden_size': 160,
    'dropout': 0.15610151695402608,
    'hidden_continuous_size': 90,
    'attention_head_size': 4,
    'learning_rate': 0.08
}

# ### summer

# PARAMS = {
#     'gradient_clip_val': 0.9658579636307634,
#     'hidden_size': 96,
#     'dropout': 0.20610151695402608,
#     'hidden_continuous_size': 90,
#     'attention_head_size': 4,
#     'learning_rate': 0.03
# }




def seed_all(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def load_dataset(train_df, validate=False):
    # Code omitted for brevity

    ENCODER_LENGTH_IN_WEEKS = 10
    max_encoder_length = 24 * 7 *ENCODER_LENGTH_IN_WEEKS #10
    max_prediction_length = 6153
    training_cutoff = train_df['time_idx'].max()-max_prediction_length #2040 - 24*7 = 1871

    tr_ds = TimeSeriesDataSet(
      train_df[lambda x: x.time_idx <=training_cutoff] if validate else train_df,
      time_idx = "time_idx",
      target = "surface_tp_train_ts", ## 예측값
      group_ids=["surface_tp_train_stn"],
      min_encoder_length = 1,
      max_encoder_length = max_encoder_length,
      min_prediction_length=1,
      max_prediction_length=max_prediction_length,
      allow_missing_timesteps=True,  # 시간 간격이 1보다 큰 경우 허용


      #Known Inputs 알고 있는 변수
      time_varying_known_categoricals = ['surface_tp_train_ww',
                                         'hh',
                                         'dd',
                                         'mm'
                                         ],
      time_varying_known_reals = [
        'surface_tp_train_ta',
        'surface_tp_train_td',
        'surface_tp_train_hm',
        'surface_tp_train_ws',
        'surface_tp_train_rn',
        'surface_tp_train_re',
        'time_idx'
        ],

      # target_normalizer=GroupNormalizer(groups=["surface_tp_train_stn"], transformation="softplus"),
      #모르고 있는 변수
      time_varying_unknown_categoricals=[],
      time_varying_unknown_reals=[
          "surface_tp_train_ts"
        ],
        add_relative_time_idx=True,  # add as feature
        add_target_scales=True,  # add as feature
        add_encoder_length=True,  # add as feature
    )

    va_ds = None
    if validate:
        va_ds = TimeSeriesDataSet.from_dataset(
        tr_ds, train_df, predict=True, stop_randomization=True
    )
    return tr_ds, va_ds

train_losses = []  # List to store training losses
val_losses = []  # List to store validation losses
def fit(seed, tr_ds, va_loader=None):
    global train_losses, val_losses
    seed_all(seed)
    ##  cache 정리
    # gc.collect()
    # torch.cuda.empty_cache()

    # Code omitted for brevity
    # training
    tr_loader = tr_ds.to_dataloader(
        train=True,
        batch_size=BATCH_SIZE,
        num_workers=4,
    )
    ### loss 찍기 추가
    if va_loader is not None:
        # stop training, when loss metric does not improve on validation set

        va_loader = va_loader.to_dataloader(
            train=False,
            batch_size=BATCH_SIZE*5,
            num_workers=4,
            shuffle = False,
        )
        checkpoint_callback = ModelCheckpoint(
            monitor='val_loss',
            dirpath=CKPTROOT,
            filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
            save_top_k=10
        )
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",
            min_delta=3e-3,
            patience=20,
            verbose=True,
            mode="min"
        )
        lr_logger = LearningRateMonitor(logging_interval="epoch")  # log the learning rate
        callbacks = [lr_logger, early_stopping_callback, checkpoint_callback]
    else:
        # gather 10 checkpoints with best traing loss
        checkpoint_callback = ModelCheckpoint(
            monitor='train_loss',
            dirpath=CKPTROOT,
            filename=f'seed={seed}'+'-{epoch:03d}-{train_loss:.2f}',
            save_top_k=10
        )
        callbacks = [checkpoint_callback]

    # create trainer
    trainer = pl.Trainer(
        max_epochs=100,
        gradient_clip_val=PARAMS['gradient_clip_val'],
        limit_train_batches=100,
        callbacks=callbacks,
        logger=TensorBoardLogger(LOGDIR),
        # train_dataloaders = tr_loader
        accelerator="gpu",
    )

    # use pre-deterined leraning rate schedule for final submission
    learning_rate = LRS if va_loader is None else PARAMS['learning_rate']

    # initialise model with pre-determined hyperparameters
    tft = TemporalFusionTransformer.from_dataset(
        tr_ds,
        learning_rate=learning_rate,
        hidden_size=PARAMS['hidden_size'],
        attention_head_size=PARAMS['attention_head_size'],
        dropout=PARAMS['dropout'],
        hidden_continuous_size=PARAMS['hidden_continuous_size'],
        output_size=1,
        loss=MAE(),  # MAE loss
        log_interval=10,  # log example every 10 batches
        logging_metrics=[MAE()],  # MAE metric
        reduce_on_plateau_patience=4,  # reduce learning automatically
    )
    print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")
    tft = tft.to('cuda:0')

    # kwargs = {'train_dataloaders': tr_loader}
    # if va_loader:
    #     kwargs['val_dataloaders'] = va_loader

    gc.collect()
    torch.cuda.empty_cache()
    # fit network
    trainer.fit(
        tft,
        train_dataloaders=tr_loader,
        val_dataloaders=va_loader,
    )

    if va_loader is not None:
      for output in trainer.callback_metrics:
          if output.startswith('train_loss'):
              train_loss = trainer.callback_metrics[output].item()
              if train_loss not in train_losses:
                train_losses.append(train_loss)
          elif output.startswith('val_loss'):
              val_loss = trainer.callback_metrics[output].item()
              val_losses.append(val_loss)

    best_model_path = trainer.checkpoint_callback.best_model_path
    print(f'best_model_path={best_model_path}')
    best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
    if va_loader is not None:
        # train_loss = trainer.callback_metrics['train_loss'].item()
        # val_loss = trainer.callback_metrics['val_loss'].item()
        print(f'Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
        return best_tft, train_loss, val_loss
    else:
        return best_tft


def main(train_df):

    # load datasets
    tr_ds, va_ds = load_dataset(train_df, validate=True)

    # fit the model
    s = 10
    model, loss_train, loss_values = fit(s, tr_ds, va_ds)

    return model, loss_train, loss_values

# # call the main function
# train_df = result_train_spring
# model, loss_train, loss_values = main(train_df)


import optuna

def objective(trial):
    # Define the hyperparameter search space
    params = {
        'gradient_clip_val': trial.suggest_loguniform('gradient_clip_val', 1e-4, 1.0),
        'hidden_size': trial.suggest_int('hidden_size', 50, 200),
        'dropout': trial.suggest_uniform('dropout', 0.1, 0.5),
        'hidden_continuous_size': trial.suggest_int('hidden_continuous_size', 50, 200),
        'attention_head_size': trial.suggest_int('attention_head_size', 1, 8),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1.0)
    }

    # Set the parameters for training
    PARAMS = params

    train_df = result_train_spring
    # Call the main function with the updated parameters
    model, loss_train, loss_values = main(train_df)

    # Return the validation loss as the objective value to be minimized
    print(loss_train)


# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get the best parameters and objective value
best_params = study.best_params
best_value = study.best_value

print('Best Parameters:', best_params)
print('Best Objective Value:', best_value)

In [ ]:
def objective():
    # Define the hyperparameter search space
    PARAMS = {
        'gradient_clip_val': 0.9658579636307634,
        'hidden_size': 100,
        'dropout': 0.20610151695402608,
        'hidden_continuous_size': 90,
        'attention_head_size': 4,
        'learning_rate': 0.03
    }

    # Set the parameters for training
    # PARAMS = params

    train_df = result_train_spring
    # Call the main function with the updated parameters
    model, loss_train, loss_values = main(train_df)

    # Return the validation loss as the objective value to be minimized
    print(loss_train)


# Create a study object and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Get the best parameters and objective value
best_params = study.best_params
best_value = study.best_value

print('Best Parameters:', best_params)
print('Best Objective Value:', best_value)